# **全ての解析でファイルのパスは手動で変更してください**

関数定義以外の解析実行部分では、下記のようにファイルのパスを変数定義している場所があります。そこで、`dir_name1`、`dir_name2`の値(`"01_gr1"`など)を変更するとファイルのパスが変わります。

```python
# 対象とするデータフォルダ
dir_name1 = "01_gr1"
dir_name2 = "02_gr2"

# データフォルダのパスを取得
dir_pathname1 = f"./03_scanpath_csv_encoded_original_split/{dir_name1}/*"
dir_pathname2 = f"./03_scanpath_csv_encoded_original_split/{dir_name2}/*"
```

# ファイルの前処理です
時系列にデータファイルに変換するための中間csvファイルを実行するものです。上から順に実行するとうまくいきます。

In [1]:
import glob
import os
import csv
import shutil
import numpy as np
import pandas as pd

from hmmlearn import hmm
from matplotlib import pyplot as plt
import collections

n_components=2

### ディレクトリを舐めて, xlsxをcsvにしてファイルを移動する

In [4]:
dirs = glob.glob("./07_scanpath_xlsx_chi_300ms/*")

for dir in dirs:
    files = glob.glob(dir+"/*")
    for file in files:
        df = pd.read_excel(file)
        csv_filename = file.replace(".xlsx", ".csv")
        df.to_csv(csv_filename, index=False)

# 対象のディレクトリパス
src_dirs = [f"./07_scanpath_xlsx_chi_300ms/{participant}/*.csv" for participant in [1, 3, 7, 13, 14, 17]]
dest_dir = "./08_scanpath_csv_chi_300ms/01_gr1/"

existing_files = glob.glob(dest_dir + "*")
for old_file in existing_files:
    os.remove(old_file)

# 各ソースディレクトリからCSVファイルを探索し、目的のディレクトリに移動
for src_dir in src_dirs:
    csv_files = glob.glob(src_dir)
    for file in csv_files:
        # 新しいファイルパスを生成
        file_name = os.path.basename(file)
        new_path = os.path.join(dest_dir, file_name)
        print(file_name, new_path)
        # ファイルを移動
        shutil.move(file, new_path)

src_dirs = [f"./07_scanpath_xlsx_chi_300ms/{participant}/*.csv" for participant in [2, 4, 5, 6, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 23, 24]]
dest_dir = "./08_scanpath_csv_chi_300ms/02_gr2/"

existing_files = glob.glob(dest_dir + "*")
for old_file in existing_files:
    os.remove(old_file)

# 各ソースディレクトリからCSVファイルを探索し、目的のディレクトリに移動
for src_dir in src_dirs:
    csv_files = glob.glob(src_dir)
    for file in csv_files:
        # 新しいファイルパスを生成
        file_name = os.path.basename(file)
        new_path = os.path.join(dest_dir, file_name)
        print(file_name, new_path)
        # ファイルを移動
        shutil.move(file, new_path)

1-3.csv ./08_scanpath_csv_chi_300ms/01_gr1/1-3.csv
1-2.csv ./08_scanpath_csv_chi_300ms/01_gr1/1-2.csv
1-1.csv ./08_scanpath_csv_chi_300ms/01_gr1/1-1.csv
1-4.csv ./08_scanpath_csv_chi_300ms/01_gr1/1-4.csv
3-1.csv ./08_scanpath_csv_chi_300ms/01_gr1/3-1.csv
3-2.csv ./08_scanpath_csv_chi_300ms/01_gr1/3-2.csv
3-3.csv ./08_scanpath_csv_chi_300ms/01_gr1/3-3.csv
3-4.csv ./08_scanpath_csv_chi_300ms/01_gr1/3-4.csv
7-3.csv ./08_scanpath_csv_chi_300ms/01_gr1/7-3.csv
7-1.csv ./08_scanpath_csv_chi_300ms/01_gr1/7-1.csv
13-4.csv ./08_scanpath_csv_chi_300ms/01_gr1/13-4.csv
13-1.csv ./08_scanpath_csv_chi_300ms/01_gr1/13-1.csv
13-2.csv ./08_scanpath_csv_chi_300ms/01_gr1/13-2.csv
13-3.csv ./08_scanpath_csv_chi_300ms/01_gr1/13-3.csv
14-8.csv ./08_scanpath_csv_chi_300ms/01_gr1/14-8.csv
14-9.csv ./08_scanpath_csv_chi_300ms/01_gr1/14-9.csv
14-1.csv ./08_scanpath_csv_chi_300ms/01_gr1/14-1.csv
14-7.csv ./08_scanpath_csv_chi_300ms/01_gr1/14-7.csv
14-6.csv ./08_scanpath_csv_chi_300ms/01_gr1/14-6.csv
17-3.csv ./08

### csvからエンコードを行う関数

In [2]:
type = "01_gr1"

"""動作データを含める場合"""
in_csv_dirs = f"./02_scanpath_csv/{type}/*.csv"
out_csv_dirs = f"./03_scanpath_csv_encoded/{type}/*"

# """動作データを含めない場合"""
# in_csv_dirs = f"./04_scanpath_csv_no_move/{type}/*.csv"
# out_csv_dirs = f"./05_scanpath_csv_encoded_no_move/{type}/*"

# エンコードする
def find_column_with_one(df):
    # 系列データの空の配列を用意
    series_data = []
    # DataFrameの各行について処理します
    for index, row in df.iterrows():
        # 2列目以降で1が含まれている列のインデックスを取得します
        columns_with_one = [col for col in df.columns[1:] if row[col] == 1]

        # 該当の列が存在する場合は、結果を出力します
        if columns_with_one:
            # print(f"{index + 1}行目を取り出す（indexは{index}）")
            # print(f"{index + 1}行目に1があるので、{', '.join(columns_with_one)}を出力する")
            pass
        
        # 系列データを出力します
        data = ', '.join(columns_with_one)
        # series_data.append(data)

        # 系列データを記号系列に変換します
        if data == "ellipse":
            series_data.append(0)
        elif data == "ellipse1":
            series_data.append(1)
        elif data == "triangle":
            series_data.append(2)
        elif data == "triangle1":
            series_data.append(3)
        elif data == "rectangular":
            series_data.append(4)
        elif data == "rectangular1":
            series_data.append(5)
        elif data == "rectangular2":
            series_data.append(6)
        elif data == "rectangular3":
            series_data.append(6)
        elif data == "rectangular4":
            series_data.append(6)

    return series_data

# 隣り合う系列が重複していたらマージする
def merge_adjacent_duplicates(series_data):
    new_series_data = []

    # 空のリストや一要素の場合はそのまま追加
    if not series_data:
        return new_series_data
    elif len(series_data) == 1:
        return [series_data[0]]

    # 最初の要素を新しいリストに追加
    new_series_data.append(series_data[0])

    # 隣り合う要素が重複していない場合は新しいリストに追加
    for i in range(1, len(series_data)):
        if series_data[i] != series_data[i - 1]:
            new_series_data.append(series_data[i])

    return new_series_data


files = glob.glob(in_csv_dirs)

for filename in files:
    print(filename)
    df = pd.read_csv(filename)

    new_filename = filename.replace(".csv", "_encoded")
    
    # 関数を実行して結果を系列データとして出力します
    series_data = find_column_with_one(df)

    # 隣り合う要素が重複していない場合にまとめる
    # series_data = merge_adjacent_duplicates(series_data)

    df_encoded = pd.DataFrame(series_data)

    df_encoded.to_csv(new_filename+".csv", index=False)


# ファイル名に"_encoded"を含むファイルを削除
file_dirs_encoded = glob.glob(out_csv_dirs)

for file in file_dirs_encoded:
    if "_encoded" in file:
        os.remove(file)

encoded_files = glob.glob(in_csv_dirs)

for encoded_file in encoded_files:
    if "encoded" in encoded_file:
        move_file_dirs = out_csv_dirs.replace("*","")
        new_path = shutil.move(encoded_file, move_file_dirs)

./02_scanpath_csv/01_gr1/2-2.csv
./02_scanpath_csv/01_gr1/4-4.csv
./02_scanpath_csv/01_gr1/2-3.csv
./02_scanpath_csv/01_gr1/2-1.csv
./02_scanpath_csv/01_gr1/2-4.csv
./02_scanpath_csv/01_gr1/4-2.csv
./02_scanpath_csv/01_gr1/4-3.csv
./02_scanpath_csv/01_gr1/4-1.csv
./02_scanpath_csv/01_gr1/12-4.csv
./02_scanpath_csv/01_gr1/8-1.csv
./02_scanpath_csv/01_gr1/12-5.csv
./02_scanpath_csv/01_gr1/8-3.csv
./02_scanpath_csv/01_gr1/8-2.csv
./02_scanpath_csv/01_gr1/12-2.csv
./02_scanpath_csv/01_gr1/8-6.csv
./02_scanpath_csv/01_gr1/12-3.csv
./02_scanpath_csv/01_gr1/12-1.csv
./02_scanpath_csv/01_gr1/8-5.csv
./02_scanpath_csv/01_gr1/8-4.csv
./02_scanpath_csv/01_gr1/9-3.csv
./02_scanpath_csv/01_gr1/9-2.csv
./02_scanpath_csv/01_gr1/9-1.csv
./02_scanpath_csv/01_gr1/9-5.csv
./02_scanpath_csv/01_gr1/9-4.csv


In [12]:
# 対象のディレクトリパス
src_dirs = ["./03_scanpath_csv_encoded/01_gr1/*.csv", "./03_scanpath_csv_encoded/03_gr2+fig/*.csv"]
dest_dir = "./03_scanpath_csv_encoded/06_fig/"

existing_files = glob.glob(dest_dir + "*")
for old_file in existing_files:
    os.remove(old_file)

# 各ソースディレクトリからCSVファイルを探索し、目的のディレクトリに移動
for src_dir in src_dirs:
    csv_files = glob.glob(src_dir)
    for file in csv_files:
        # 新しいファイルパスを生成
        file_name = os.path.basename(file)
        new_path = os.path.join(dest_dir, file_name)
        print(file_name, new_path)
        # ファイルを移動
        shutil.move(file, new_path)

# 対象のディレクトリパス
src_dirs = ["./03_scanpath_csv_encoded/04_gr2+non_fig/*.csv"]
dest_dir = "./03_scanpath_csv_encoded/07_non_fig/"

existing_files = glob.glob(dest_dir + "*")
for old_file in existing_files:
    os.remove(old_file)

# 各ソースディレクトリからCSVファイルを探索し、目的のディレクトリに移動
for src_dir in src_dirs:
    csv_files = glob.glob(src_dir)
    for file in csv_files:
        # 新しいファイルパスを生成
        file_name = os.path.basename(file)
        new_path = os.path.join(dest_dir, file_name)
        print(file_name, new_path)
        # ファイルを移動
        shutil.move(file, new_path)

8-6_encoded.csv ./03_scanpath_csv_encoded/06_fig/8-6_encoded.csv
9-1_encoded.csv ./03_scanpath_csv_encoded/06_fig/9-1_encoded.csv
4-2_encoded.csv ./03_scanpath_csv_encoded/06_fig/4-2_encoded.csv
9-4_encoded.csv ./03_scanpath_csv_encoded/06_fig/9-4_encoded.csv
12-2_encoded.csv ./03_scanpath_csv_encoded/06_fig/12-2_encoded.csv
8-3_encoded.csv ./03_scanpath_csv_encoded/06_fig/8-3_encoded.csv
2-3_encoded.csv ./03_scanpath_csv_encoded/06_fig/2-3_encoded.csv
12-1_encoded.csv ./03_scanpath_csv_encoded/06_fig/12-1_encoded.csv
4-1_encoded.csv ./03_scanpath_csv_encoded/06_fig/4-1_encoded.csv
9-2_encoded.csv ./03_scanpath_csv_encoded/06_fig/9-2_encoded.csv
4-4_encoded.csv ./03_scanpath_csv_encoded/06_fig/4-4_encoded.csv
12-4_encoded.csv ./03_scanpath_csv_encoded/06_fig/12-4_encoded.csv
8-5_encoded.csv ./03_scanpath_csv_encoded/06_fig/8-5_encoded.csv
2-1_encoded.csv ./03_scanpath_csv_encoded/06_fig/2-1_encoded.csv
12-3_encoded.csv ./03_scanpath_csv_encoded/06_fig/12-3_encoded.csv
8-2_encoded.csv .

In [6]:
type = "01_gr1"

in_csv_dirs = f"./08_scanpath_csv_chi_300ms/{type}/*.csv"
out_csv_dirs = f"./09_scanpath_csv_encoded_chi_300ms/{type}/*"


# エンコードする
def find_column_with_one(df):
    # 系列データの空の配列を用意
    series_data = []
    # DataFrameの各行について処理します
    for index, row in df.iterrows():
        # 2列目以降で1が含まれている列のインデックスを取得します
        columns_with_one = [col for col in df.columns[1:] if row[col] == 1]

        # 該当の列が存在する場合は、結果を出力します
        if columns_with_one:
            # print(f"{index + 1}行目を取り出す（indexは{index}）")
            # print(f"{index + 1}行目に1があるので、{', '.join(columns_with_one)}を出力する")
            pass
        
        # 系列データを出力します
        data = ', '.join(columns_with_one)
        # series_data.append(data)

        # 系列データを記号系列に変換します
        if data == "ellipse":
            series_data.append(0)
        elif data == "ellipse1":
            series_data.append(1)
        elif data == "triangle":
            series_data.append(2)
        elif data == "triangle1":
            series_data.append(3)
        elif data == "rectangular":
            series_data.append(4)
        elif data == "rectangular1":
            series_data.append(5)
        elif data == "rectangular2":
            series_data.append(6)
        elif data == "rectangular3":
            series_data.append(6)
        elif data == "rectangular4":
            series_data.append(6)

    return series_data

# 隣り合う系列が重複していたらマージする
def merge_adjacent_duplicates(series_data):
    new_series_data = []

    # 空のリストや一要素の場合はそのまま追加
    if not series_data:
        return new_series_data
    elif len(series_data) == 1:
        return [series_data[0]]

    # 最初の要素を新しいリストに追加
    new_series_data.append(series_data[0])

    # 隣り合う要素が重複していない場合は新しいリストに追加
    for i in range(1, len(series_data)):
        if series_data[i] != series_data[i - 1]:
            new_series_data.append(series_data[i])

    return new_series_data


files = glob.glob(in_csv_dirs)

for filename in files:
    print(filename)
    df = pd.read_csv(filename)

    new_filename = filename.replace(".csv", "_encoded")
    
    # 関数を実行して結果を系列データとして出力します
    series_data = find_column_with_one(df)

    # 隣り合う要素が重複していない場合にまとめる
    # series_data = merge_adjacent_duplicates(series_data)

    df_encoded = pd.DataFrame(series_data)

    df_encoded.to_csv(new_filename+".csv", index=False)


# ファイル名に"_encoded"を含むファイルを削除
file_dirs_encoded = glob.glob(out_csv_dirs)

for file in file_dirs_encoded:
    if "_encoded" in file:
        os.remove(file)

encoded_files = glob.glob(in_csv_dirs)

for encoded_file in encoded_files:
    if "encoded" in encoded_file:
        move_file_dirs = out_csv_dirs.replace("*","")
        new_path = shutil.move(encoded_file, move_file_dirs)

./08_scanpath_csv_chi_300ms/02_gr2/20-1.csv
./08_scanpath_csv_chi_300ms/02_gr2/22-3.csv
./08_scanpath_csv_chi_300ms/02_gr2/24-4.csv
./08_scanpath_csv_chi_300ms/02_gr2/22-2.csv
./08_scanpath_csv_chi_300ms/02_gr2/19-4.csv
./08_scanpath_csv_chi_300ms/02_gr2/20-2.csv
./08_scanpath_csv_chi_300ms/02_gr2/5-4.csv
./08_scanpath_csv_chi_300ms/02_gr2/22-1.csv
./08_scanpath_csv_chi_300ms/02_gr2/20-3.csv
./08_scanpath_csv_chi_300ms/02_gr2/19-3.csv
./08_scanpath_csv_chi_300ms/02_gr2/5-1.csv
./08_scanpath_csv_chi_300ms/02_gr2/22-5.csv
./08_scanpath_csv_chi_300ms/02_gr2/24-3.csv
./08_scanpath_csv_chi_300ms/02_gr2/24-2.csv
./08_scanpath_csv_chi_300ms/02_gr2/22-4.csv
./08_scanpath_csv_chi_300ms/02_gr2/19-2.csv
./08_scanpath_csv_chi_300ms/02_gr2/20-6.csv
./08_scanpath_csv_chi_300ms/02_gr2/20-4.csv
./08_scanpath_csv_chi_300ms/02_gr2/5-2.csv
./08_scanpath_csv_chi_300ms/02_gr2/24-1.csv
./08_scanpath_csv_chi_300ms/02_gr2/11-8.csv
./08_scanpath_csv_chi_300ms/02_gr2/5-3.csv
./08_scanpath_csv_chi_300ms/02_gr2/1